In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os
os.chdir("/content/drive/My Drive/DL/Lesson13/thuchanh")

Mounted at /content/drive


**Nội dung bài thực hành**

Người học tiếp cận và giải quyết bài toán phân lớp văn bản sử dụng Deep Learning với kiến trúc LSTM. Sau khi thực hành, người học có khả năng:
1. Sử dụng được các công cụ của sklearn cho bài toán phân lớp
  

*   Thao tác với dữ liệu
*   Chuyển từ văn bản sang không gian vector


2. Áp dụng được Deep Learning cho bài toán phân lớp


*   Huấn luyện mô hình
*   Đánh gía mô hình


3. Cải tiến được mô hình phân lớp
3. Thực hành được với bài toán thực tế

**Thao tác với dữ liệu**



*   Bài thực hành sử dụng dữ liệu tiếng Anh
*   Dữ liệu gồm 18.000 bài báo được tổ chức trong 20 lớp (classes/groups)
*   Yêu cầu: xây dựng mô hình phân lớp các bài báo dựa trên mô hình LSTM



In [ ]:
#Loading the data set - training data.
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)

In [ ]:
len(twenty_test.data)

7532

**Quan sát dữ liệu huấn luyện**



*   Số lượng dữ liệu huấn luyện
*   Quan sát một số bản ghi đầu tiên





*   Xem danh sách tên các lớp bằng câu lệnh dưới đây

In [ ]:
# YOUR CODE HERE
print("Ten cac lop", twenty_train.target_names)

Ten cac lop ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']




*   Nhãn của các lớp



In [ ]:
# YOUR CODE HERE
print("Nhan cac lop", twenty_train.target)

Nhan cac lop [7 4 4 ... 3 1 8]




*  Hiển thị dòng đầu tiên của văn bản đầu tiên



In [ ]:
print("\n".join(twenty_train.data[0].split("\n")[:3])) #prints first line of the first data file

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu


#**Câu 1**: Tiền xử lý dữ liệu



1.1: Lập trình hàm preprocessor để loại bỏ những ký tự thừa ra khỏi văn bản

In [ ]:
import re
# Extracting features from text files
# YOUR CODE HERE
def clean_info(text):
  # YOUR CODE HERE
  text = "".join(text.split("\n")[1:])
  return text
def preprocessor(text):
    
    text = re.sub(r"<[^>]>", '', text)
    text = re.sub("[\W]+", " ", text.lower())

    return text.lower().strip()

def preprocess_whole_data(docs):
  # YOUR CODE HERE
  processed_text = [preprocessor(clean_info(data)) for data in docs.data]
  return processed_text

# YOUR CODE HERE
preprocessor(clean_info(twenty_train.data[0]))

'subject what car is this nntp posting host rac3 wam umd eduorganization university of maryland college parklines 15 i was wondering if anyone out there could enlighten me on this car i sawthe other day it was a 2 door sports car looked to be from the late 60s early 70s it was called a bricklin the doors were really small in addition the front bumper was separate from the rest of the body this is all i know if anyone can tellme a model name engine specs yearsof production where this car is made history or whatever info youhave on this funky looking car please e mail thanks il brought to you by your neighborhood lerxst'

1.2. Tiền xử lý cho cả tập train và test

In [ ]:
# YOUR CODE HERE
process_train_data = preprocess_whole_data(twenty_train)
process_test_data = preprocess_whole_data(twenty_test)
# YOUR CODE HERE

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM # comment dong nay neu may ban khong ho tro CuDNN
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, TimeDistributed, BatchNormalization
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K



# embedding
word2vec_file = 'GoogleNews-vectors-negative300.bin'
glove_file = 'glove.6B.300d.txt'
fasttext_file = 'wiki-news-300d-1M.vec'

1.3: Biến đổi từ về dạng token trong câu

In [ ]:
# YOUR CODE HERE
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(process_train_data)
X_train = tokenizer.texts_to_sequences(process_train_data)
X_test = tokenizer.texts_to_sequences(process_test_data)
X_train = pad_sequences(X_train, maxlen=500)
X_test = pad_sequences(X_test, maxlen=500)
# YOUR CODE HERE

1.4: Biến đổi nhãn về dạng One-hot Vector

In [ ]:
# YOUR CODE HERE
train_one_hot = to_categorical(twenty_train.target.reshape(-1,1))
test_one_hot = to_categorical(twenty_test.target.reshape(-1,1))

# YOUR CODE HERE


# **Câu 2:** Xây dựng mô hình
2.1. Thiết lập tham số

In [ ]:

# YOUR CODE HERE
max_len = 300
num_words = 5000
embedding_dim = 300
dropout = 0.5
hidden_dim = 750
l2_reg = 1e-4
batch_size = 160
epochs = 15
learning_rate = 1e-3
lstm = CuDNNLSTM
opt = 'adam'
# YOUR CODE HERE

2.2. Biểu diễn văn bản bằng Word2vec, Glove, và Fasttext



In [ ]:
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText
import numpy as np

def get_embedded(wordvector):
    # YOUR CODE HERE
    word_exits = 0
    vocab = tokenizer.index_word
    embedded_matrix = np.zeros((5000,300))
    print("Doc wordvector to %s ..." %wordvector)
    if wordvector == "word2vec":
        model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    elif wordvector == "glove":
        model = {}
        with open('glove.6B.300d.txt', 'r') as f:
            for line in f:
                l = line.split()
                word = l[0]
                model[word] = np.array(l[1:]).astype(float)
    elif wordvector == "fasttext":
        model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')

    for word in vocab:
        if word > 4999:
            break
        try:
            embedded_matrix[word, :] = model[vocab[word]]
            word_exits += 1
        except KeyError:
            if word == 0:
                embedded_matrix[word, :] = np.zeros(300)
            else:
                # 0.25 is embedding SD
                embedded_matrix[word, :] = np.random.uniform(-0.25, 0.25, 300)
    print("Found %s word in embedding file" %word_exits)
    # YOUR CODE HERE
    return embedded_matrix

2.3. Lập trình hàm `get_model()` để xây dựng kiến trúc mạng Deep Learning cho bài toán. Học viên có thể tham khảo mô hình của bài thực hành trên lớp

In [ ]:
def get_model(embedding="random"):
    # YOUR CODE HERE
    input_s = Input(name='input_si', shape=(max_len,), dtype='int32')
    if embedding == "random":
        shared_embedding = Embedding(name='emb', input_dim=num_words, input_length=max_len,
                                output_dim=embedding_dim, mask_zero=False, trainable=True)
    else:
        embedded_matrix = get_embedded(embedding)
        shared_embedding = Embedding(name='emb', input_dim=num_words, input_length=max_len,
                                     weights=[embedded_matrix],output_dim=embedding_dim, mask_zero=False, trainable=True)
    s_embedding = shared_embedding(input_s)
    s_embedding = Dropout(dropout)(s_embedding)
    shared_lstm = Bidirectional(
            lstm(hidden_dim, input_shape=(None, 500, 300),return_sequences=False, name='rnn'),
            merge_mode='concat')
    s_lstm = shared_lstm(s_embedding)
    s_lstm = Dropout(dropout)(s_lstm)
    yhat = Dense(20, activation="sigmoid")(s_lstm)
    model = Model(inputs=input_s, outputs=yhat)
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer="adam")
    # YOUR CODE HERE
    return model

In [ ]:
# YOUR CODE HERE
text_clf = get_model()
# YOUR CODE HERE
text_clf.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_si (InputLayer)        [(None, 300)]             0         
_________________________________________________________________
emb (Embedding)              (None, 300, 300)          1500000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 300, 300)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1500)              6312000   
_________________________________________________________________
dropout_7 (Dropout)          (None, 1500)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                30020     
Total params: 7,842,020
Trainable params: 7,842,020
Non-trainable params: 0
____________________________________________

# **Câu 3:** Huấn luyện mô hình



3.1. Khởi tạo mô hình



In [ ]:
# YOUR CODE HERE
text_clf = get_model()
# YOUR CODE HERE
text_clf.summary()

Doc wordvector to glove ...
Found 4895 word in embedding file
Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_si (InputLayer)        [(None, 300)]             0         
_________________________________________________________________
emb (Embedding)              (None, 300, 300)          1500000   
_________________________________________________________________
dropout_14 (Dropout)         (None, 300, 300)          0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 750)               6312000   
_________________________________________________________________
activation_7 (Activation)    (None, 750)               0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 750)               0         
_________________________________________________________

3.2. Huấn luyện mô hình

In [ ]:
text_clf.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer= tf.keras.optimizers.Adam(learning_rate = learning_rate))
text_clf.fit(X_train, train_one_hot, validation_split=0.2, batch_size=batch_size, epochs=11)

Epoch 1/11
57/57 [==============================] - 77s 1s/step - loss: 2.9714 - accuracy: 0.0835 - val_loss: 2.9258 - val_accuracy: 0.0941
Epoch 2/11
57/57 [==============================] - 66s 1s/step - loss: 2.6841 - accuracy: 0.1543 - val_loss: 2.4712 - val_accuracy: 0.2311
Epoch 3/11
57/57 [==============================] - 68s 1s/step - loss: 2.3547 - accuracy: 0.2583 - val_loss: 2.1155 - val_accuracy: 0.2881
Epoch 4/11
57/57 [==============================] - 68s 1s/step - loss: 1.8840 - accuracy: 0.3853 - val_loss: 2.0199 - val_accuracy: 0.3535
Epoch 5/11
57/57 [==============================] - 69s 1s/step - loss: 1.4679 - accuracy: 0.5181 - val_loss: 1.6561 - val_accuracy: 0.4795
Epoch 6/11
57/57 [==============================] - 70s 1s/step - loss: 1.2217 - accuracy: 0.5973 - val_loss: 1.6851 - val_accuracy: 0.4759
Epoch 7/11
57/57 [==============================] - 70s 1s/step - loss: 1.0048 - accuracy: 0.6700 - val_loss: 1.7652 - val_accuracy: 0.4843
Epoch 8/11
57/57 [==

In [ ]:
text_clf.evaluate(X_test,test_one_hot)

236/236 [==============================] - 28s 120ms/step - loss: 2.1812 - accuracy: 0.4521


[2.1812267303466797, 0.4520711600780487]